In [1]:
from osgeo import ogr
from osgeo import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
import critical_loads as cl
from IPython.display import Image
import imp

# Critical loads for vegetation

## 1. Mosaic 30 m land cover data

Raw land cover data with 30 m resolution are here:

K:\Avdeling\317 Klima- og miljømodellering\KAU\Focal Centre\Vegetation\Veg map\satveg_30\0

I've copied this locally and used the `Mosiac_To_New_Raster` tool in ArcToolbox to combine the tiles into a single 8-bit integer GeoTiff (`sat_veg_30m_all.tif`), which has an uncompressed size of 2.2 GB.

## 2. Reclassify vegetation according to critical loads

### 2.1. Read lookup table link vegetation classes to critical loads

The land use classes for the vegeation map are given here:

C:\Data\James_Work\Staff\Kari_A\Critical_Loads\sat_veg_land_use_classes.xlsx

The first step is to reclassify the land use grid according to the critical loads. Land classes 0, 23, 24 and 25 are not used, so I'll set the values to 255 (for no data).

In [2]:
# Read lookup table
in_xlsx = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\sat_veg_land_use_classes.xlsx')
df = pd.read_excel(in_xlsx, sheetname='EUNIS_tilGIS', index_col=0)

df = df[['CL_meq/m2/yr']].round(0).astype(int)

df

,CL_meq/m2/yr
NORUTcode,
1,36
2,36
3,36
4,71
5,71
6,36
7,36
8,36
9,36


### 2.2. Reclassify

In [3]:
# Input national veg map
in_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_all.tif')

# Output geotiff for critical loads values
out_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\GIS\Raster\sat_veg_30m_cr_lds.tif')

# Reclassify
cl.reclassify_raster(in_tif, out_tif, df, 'CL_meq/m2/yr', 255)

## 3. Process deposition data

### 3.1. Get deposition data from database

The deposition data are found in `RESA2.DEP_BLR_VALUES`. Parameter IDs are defined in `RESA2.AIR_PARAMETER_DEFINITIONS` and the various data series are defined in `RESA2.DEP_SERIES_DEFINITIONS`. For vegetation, we calculate exceedance as

$$E_{veg} = Dep_N - CL$$

where $Dep_N$ is total nitrogen deposition and $CL$ is the critical load (see e-mail from Espen received  03/10/2017 at 14.25). Parameter ID 6 in `RESA2.AIR_PARAMETER_DEFINITIONS` corresponds to `ENTot` in `mEkv/m2/year`, which I assume is what we use for $Dep_N$ in the equation above. The different time series definitions are less obvious to me, and I'm not sure which ones we need to recalculate this year. I have exported this table to:

C:\Data\James_Work\Staff\Kari_A\Critical_Loads\dep_series_names.xlsx

**Ask Kari/Espen which series we are interested in**. For now, I'll just pick a series at random for use in testing.

In [4]:
# Connect to db
resa2_basic_path = (r'C:\Data\James_Work\Staff\Heleen_d_W\ICP_Waters\Upload_Template'
                    r'\useful_resa2_code.py')
resa2_basic = imp.load_source('useful_resa2_code', resa2_basic_path)
engine, conn = resa2_basic.connect_to_resa2()

In [5]:
# Get all dep values for ENTot (par_id = 6)
sql = ("SELECT a.blr, b.name, ROUND(a.value) as dep "
       "FROM resa2.dep_blr_values a, "
       "resa2.dep_series_definitions b "
       "WHERE a.dep_series_id = b.dep_series_id "
       "AND a.parameter_id = 6")

df = pd.read_sql(sql, engine)

# Convert to 'wide' format
df.set_index(['blr', 'name'], inplace=True)
df = df.unstack()
df.columns = df.columns.get_level_values(1)
df.columns.name = ''

df.head()

,"2010 (EMEP/CCE 2008) ""his""","2020 (EMEP/CCE 2008) ""CLE""","2020 (EMEP/CCE 2008) ""MFR""",Call 2008 CLE2020,Call 2008 HIS2000,Call 2008 HIS2010,Call 2008 MFR2020,expost gridav MFR2020,expost gridav MFR2020-Update,expost gridav NAT2000,expost gridav NAT2000-Update,expost gridav NAT2020,expost gridav NAT2020-Update,expost gridav PRI2020,expost gridav PRI2020-Update,expost gridav PRI2030,expost gridav PRI2030-Update
blr,,,,,,,,,,,,,,,,,
58005004,NaN,NaN,NaN,45.0,61.0,49.0,27.0,21.0,23.0,53.0,62.0,37.0,44.0,36.0,42.0,35.0,41.0
58006001,58.0,52.0,31.0,49.0,67.0,54.0,29.0,22.0,26.0,56.0,68.0,39.0,48.0,37.0,46.0,36.0,45.0
58006002,58.0,53.0,31.0,54.0,75.0,60.0,31.0,24.0,28.0,62.0,75.0,42.0,52.0,41.0,50.0,39.0,49.0
58006003,58.0,53.0,31.0,54.0,75.0,60.0,31.0,24.0,28.0,62.0,75.0,42.0,52.0,41.0,50.0,39.0,49.0
58006004,92.0,84.0,49.0,54.0,75.0,60.0,31.0,24.0,28.0,62.0,75.0,42.0,52.0,41.0,50.0,39.0,49.0


In [6]:
# Get one column as an example
dep_df = df[['Call 2008 MFR2020']].dropna(how='any').astype(int).reset_index()

# Rename cols to match shapefile
dep_df.columns = ['BLR', 'MFR2020']

dep_df.head()

,BLR,MFR2020
0,58005004,27
1,58006001,29
2,58006002,31
3,58006003,31
4,58006004,31


### 3.2. Join to BLR grid

The deposition values from the dataabse need to be joined to the BLR grid shapefile.

In [7]:
# Read BLR grid
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums_utm_z33n.shp')

blr_df = gpd.read_file(in_shp)

# Join
dep_df = blr_df.merge(dep_df, on='BLR')
del dep_df['area_m2']

dep_df.head()

,BLR,geometry,MFR2020
0,58005004,(POLYGON ((-25471.98130414146 6514836.99841211...,27
1,58006001,POLYGON ((-10886.67732153146 6503779.108531611...,29
2,58006002,"POLYGON ((3660.479678658361 6501900.217185441,...",31
3,58006003,"POLYGON ((18211.98529702786 6500076.037824233,...",31
4,58006004,"POLYGON ((32767.70791637653 6498306.548390091,...",31


In [8]:
# Write to shapefile
out_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
           r'\Shapefiles\dep_shape.shp')

dep_df.to_file(out_shp)

### 3.3. Convert BLR grid to 30 m raster

We can now convert the shapefile to a 30 m raster with exactly the same extent etc. as the reclassified vegetation map. As with the vegetation and critical loads maps, all values are stored as 8-bit (= 1 byte) integers to keep file sizes down

In [9]:
# BLR grid
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Shapefiles\dep_shape.shp')

# Output BLR raster
out_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\GIS\Raster\dep_30m.tif')

# Snap raster
snap_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\sat_veg_30m_all.tif')

# Rasterize
cl.vec_to_ras(in_shp, out_tif, snap_tif, 'MFR2020', 255, gdal.GDT_Byte)

## 4. Calculate exceedance

Exceedance can now be calculated easily by subtracting these grids. Some care is need to properly handle no data values and to avoid arithmetic over-/under-flow. In the code below, I temporarily upcast both grids to 32-bit floats. This is not very memory efficient, but it's easy and with 32 GB of RAM on my laptop it should be fine.

In [10]:
# Paths to CL and DEP grids
cl_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_cr_lds.tif')

dep_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\GIS\Raster\dep_30m.tif')

# Read grids
cl_grid, cl_ndv = cl.read_geotiff(cl_tif)
dep_grid, dep_ndv = cl.read_geotiff(dep_tif)

# Upcast to float32 for safe handling of negative values
cl_grid = cl_grid.astype(np.float32)
dep_grid = dep_grid.astype(np.float32)

# Set ndv
cl_grid[cl_grid==cl_ndv] = np.nan
dep_grid[dep_grid==dep_ndv] = np.nan

# Exceedance
ex_grid = dep_grid - cl_grid
del dep_grid, cl_grid

# Reset ndv
ex_grid[np.isnan(ex_grid)] = 255

# Set <0 to 0
ex_grid[ex_grid<0] = 0

# Downcast to uint8 to save space
ex_grid = ex_grid.astype(np.uint8)

In [11]:
# Output exceedance
out_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\GIS\Raster\exceed_mfr2020.tif')

# Snap raster
snap_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\sat_veg_30m_all.tif')

# Write results
cl.write_geotiff(ex_grid, out_tif, snap_tif, 255, gdal.GDT_Byte)

This whole workflow takes approximately 5 minutes to run, which I think is much faster than the previous vector-based approach. Once I know which deposition series we are interested in, I can restructure the code to loop over all the datasets.

An ArcMap file showing the results from this experiemnt can be found on the network here:

K:\Prosjekter\JES\Critical_Loads